## Open Government Data, provided by **OpenDataZurich**
*Autogenerated Python starter code for data set with identifier* **geo_bauernhofladen**

## Dataset
# **Bauernhofladen**

## Description

Beinhaltet die Betriebszentren von Haupterwerbslandwirten in der Stadt Zürich inklusive der von der Quartierbevölkerung betriebenen Bauernhof Weinegg.


**Zweck**: Für den Digitalen Stadtplan werden die Standorte aller Bauernhöfe mit Direktvermarktung benötigt. Für interne Projekte (Vernetzungsprojekt, Betriebszentren-Karte, usw.) werden auch die übrigen Bauernhöfe benötigt.

**Genereller Hinweis zum Geodatensatz:**

Es handelt sich hierbei um einen Geodatensatz, welcher aus mehreren Geodatenelmenten besteht. Diese Geodatenelemente werden hier nicht im Detail beschrieben. Sie finden jedoch sämtliche Informationen - wie beispielsweise die Attributbeschreibungen - [«**Bauernhofladen»** auf Geocat.ch](https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/232622bc-15e4-4701-bfcb-5b6f005f8d66).
			   

**Informationen zum Datensatz:**

Zusammenfassung:
Der Datensatz enthält die Betriebszentren von Haupterwerbslandwirten in der Stadt Zürich sowie dem von der Quartierbevölkerung betriebenen Bauernhof Weinegg. Es wird unterschieden zwischen städtischen und privaten Betrieben, sowie solchen mit und ohne Direktvermarktung. Der Datenbestand ist im Digitalen Stadtplan zugänglich und wird zur Erstellung diverser interner Karten benötigt.

Bemerkung zur Nachführung:
Nur selten Mutation nötig, da selten ein Betrieb aufgelöst bzw. eröffnet wird.
Die Nachführung geschieht dabei innerhalb des von GeoZ betreuten Werkzeugs 'POI-Verwaltung'.


**Datenerfassung:**

Die Geometrie wurde aus dem Hausnummern-Datenbestand der amtlichen Vermessung übernommen. Die Attribute werden aufgrund der Fachkenntnisse der Datennachführenden sowie Befragung der BetriebsleiterInnen erfasst.

**Datengrundlage:**

Daten der amtlichen Vermessung (Digitalisier-Grundlage, Attribut-Quelle) Hausnummern als Standorts-Grundlage

**Weitere Informationen zu diesem Thema:**

Finden Sie unter diesem [**Link**](http://www.stadt-zuerich.ch/content/ted/de/index/gsz/natur-_und_erlebnisraeume/landwirtschaft_in_der_stadt_zuerich/direktvermarktung.html)

**Geodatenelemente:**

Die vollständige Metadatenbeschreibung der Geodatenelemente finden Sie ebenfalls unter [«**Bauernhofladen»** auf Geocat.ch](https://www.geocat.ch/geonetwork/srv/ger/md.viewer#/full_view/232622bc-15e4-4701-bfcb-5b6f005f8d66).

***POI_BAUERNHOFLADEN_VIEW***: 

Zugriff über ArcMap für GSZ-Mitarbeitende: Geoserver-Daten-Menü -&gt; Inventare/Ist-Zustand GSZ -&gt; Bauernhöfe


***POI_BAUERNHOFLADEN_ATT***: 

Zusatzinformation Bauernhofläden




**Statisches Vorschaubild:**

![BildText](https://www.gis.stadt-zuerich.ch/zueriplan_docs/geocat/232622bc-15e4-4701-bfcb-5b6f005f8d66.png)



## Data set links

[Direct link by OpenDataZurich for dataset](https://data.stadt-zuerich.ch/dataset/geo_bauernhofladen)

https://www.stadt-zuerich.ch/geodaten/download/Bauernhofladen?format=geojson_link<br>


## Metadata
- **Publisher** `Landwirtschaft, Grün Stadt Zürich, Tiefbau- und Entsorgungsdepartement`
- **Maintainer** `Open Data Zürich`
- **Maintainer_email** `opendata@zuerich.ch`
- **Keywords** `Bevölk­erung`
- **Tags** `['bauernhof', 'geodaten', 'geoportal', 'landwirtschaft', 'punktdaten', 'stzh']`
- **Metadata_created** `2023-11-06T03:15:14.251457`
- **Metadata_modified** `2025-06-21T06:36:04.687036`


## Imports and helper functions

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 

import json
import xml.etree.ElementTree as ET
import requests
import geopandas as gpd
import re

In [ ]:
# helper function for reading datasets with proper separator
def get_dataset(url):
    # get URL ofs WFS service
    dataset_identifier = re.search(r'\/([^\/\?]+)\?', url).group(1)
    url_geoportal = f"https://www.ogd.stadt-zuerich.ch/wfs/geoportal/{dataset_identifier}"
    print("Getting available layers from:", url_geoportal)
    
    # Parameter for GetCapabilities
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetCapabilities"
    }
    
    # send GetCapabilities
    response = requests.get(url_geoportal, params=params)

    # parse XML answer
    root = ET.fromstring(response.content)

    # define Namespace 
    namespace = {'wfs': 'http://www.opengis.net/wfs'}
    
    # Exctract available layers
    layers = [feature_type.find('wfs:Name', namespace).text for feature_type in root.findall('.//wfs:FeatureType', namespace)]
    
    print("Available layers:", layers)
    print("First layer is set as default. To chose another layer set it as typename in the get_dataset() function.")

    # set first layer als typename
    typename = layers[0]
    print("Chosen typename:", typename)

    # Parameter GetFeature request
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetFeature",
        "typename": typename,
        "outputFormat": "application/json"
    }
    
    # GetFeature request
    response = requests.get(url_geoportal, params=params)

    # Load GeoJSON in GeoDataFrame
    gdf = gpd.read_file(json.dumps(response.json()))
    return gdf

## Load the data

In [ ]:
gdf = get_dataset('https://www.stadt-zuerich.ch/geodaten/download/Bauernhofladen?format=geojson_link')

## Analyze the data

In [ ]:
gdf.plot()

In [ ]:
# drop columns that have no values
gdf.dropna(how='all', axis=1, inplace=True)

In [ ]:
print(f'The dataset has {gdf.shape[0]:,.0f} rows (observations) and {gdf.shape[1]:,.0f} columns (variables).')
print(f'There seem to be {gdf.duplicated().sum()} exact duplicates in the data.')

In [ ]:
gdf.info(memory_usage='deep', verbose=True)

In [ ]:
gdf.head()

In [ ]:
# display a small random sample transposed in order to see all variables
gdf.sample(3).T

In [ ]:
# describe non-numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(exclude='number'))
except:
    print("No categorical data in dataset.")

In [ ]:
# describe numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(include='number'))
except:
    print("No numercial data in dataset.")

In [ ]:
# plot a histogram for each numerical feature
try:
    gdf.hist(bins=25, rwidth=.9)
    plt.tight_layout()
    plt.show()
except:
    print("No numercial data to plot.") 

In [ ]:
# continue your code here...

**Contact**: opendata@zuerich.ch